# Soccer Juggling Counter
## Manny Lazalde
### August 28, 2020

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import scipy
from scipy.signal import find_peaks
from scipy.signal import savgol_filter
import os
import time
import plotly
import plotly.express as px
import plotly.graph_objects as go
#text to speech
import pyttsx3
import multiprocessing
from threading import Thread
import threading

# custom library to make cv easier
import imutils

### Functions used to Count Ball Juggles

In [3]:
# Threading Class
# https://github.com/nateshmbhat/pyttsx3/issues/8
class Threader(Thread):
    def __init__(self, *args, **kwargs):
        Thread.__init__(self, *args, **kwargs)
        self.daemon = True
        self.start()

    def run(self):
        tts_engine = pyttsx3.init()
        tts_engine.setProperty('rate', 250)
        tts_engine.say(self._args)
        tts_engine.runAndWait()
        
# curtosy of https://github.com/Enoooooormousb
def ball_finder(frame, hsv_lower, hsv_upper):
    # blur the image to reduce noise
    blurred = cv2.GaussianBlur(frame, (11, 11), 0)
    #convert to hsv color space for color filtering
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    # construct a mask for the color specified
    mask = cv2.inRange(hsv, hsv_lower, hsv_upper)
    # eliminate low color signal 
    mask = cv2.erode(mask, None, iterations=2)
    # expand the strong color signal
    mask = cv2.dilate(mask, None, iterations=2)
    return mask

#count number of peaks in trace so far. Returns # of peaks
def peak_calculator(height,cur_num_peaks):
    if len(height) > 9:
        #invert and filter input
        y = savgol_filter(height,9,2)
        #use peak finder
        peaks, _ = find_peaks(y)
        if len(peaks) < cur_num_peaks:
            peaks = [cur_num_peaks]
    else:
        peaks = [0]
    return str(len(peaks))

In [4]:
path = os.getcwd()

#HSV color limits of ball
#https://stackoverflow.com/questions/10948589/choosing-the-correct-upper-and-lower-hsv-boundaries-for-color-detection-withcv
colorLower = np.array([25,80,20])
colorUpper = np.array([40,180,255])

#location of video file
video_path = path + '\\Soccer Juggling Data\\Outside_Sun.mp4'


### Perform Video Analysis

In [26]:
#save all the center coordinates
x_centers = []
y_centers = []

# tally for speech to text
count = '0'


#Background Subtraction - pick which method to use
#backSub = cv2.createBackgroundSubtractorMOG2()
backSub = cv2.createBackgroundSubtractorKNN()
background_subtract = True


# Create a VideoCapture object and read from input file 
cap = cv2.VideoCapture(video_path) #For pre-recorded
#cap = cv2.VideoCapture(0) # for live video

# Check if camera opened successfully 
if (cap.isOpened()== False):  
    print("Error opening video file") 

# Read until video is completed
while(cap.isOpened()):
    # Capture frame-by-frame 
    ret, frame = cap.read() 
    if ret == True: 
        #resize the frame. This function keeps aspect ratio
        frame = imutils.resize(frame, width=800)
        
        #if background subtraction
        if background_subtract:
            #background subtract
            fgMask = backSub.apply(frame)
            #bitwise multiplication to grab moving part
            new_frame = cv2.bitwise_and(frame,frame, mask = fgMask)
        
        
        # identify the ball
        mask = ball_finder(new_frame, colorLower, colorUpper)

        #canny edge detection
        edges = cv2.Canny(mask,100,200)
        # find contours of the ball
        image, contours, hierarchy = cv2.findContours(edges, 
                               cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        #write frame number in top left
        cv2.rectangle(frame, (10, 2), (100,20), (255,255,255), -1)
        cv2.putText(frame, str(cap.get(cv2.CAP_PROP_POS_FRAMES)), (15, 15),
               cv2.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))
        
        #new_frame = cv2.drawContours(new_frame, contours, -1, (0,255,0), 3)
        
        # get rid of excess contours and do analysis
        if len(contours) > 0:
            # get the largest contour
            c = max(contours, key=cv2.contourArea) 
            # find the center of the ball
            M = cv2.moments(c)
            center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
            #save the center coordinates to a list
            x_centers.append(center[0])
            y_centers.append(center[1])
            peaks = peak_calculator(y_centers,int(count))
        
            #draw center and contour onto frame
            cv2.circle(frame, center, 8, (255,255,0), -1)
            img = cv2.drawContours(frame, contours, -1, (0,0,255), 2)

            #put number of peaks on frame
            cv2.putText(frame, peaks, (50,50), 
                        cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,0,255), 2)

            #say outloud the count    
            if count != peaks:
                count = peaks
                if int(count) % 2 == 0:
                    my_thread = Threader(args = peaks)

        # Display the resulting frame 
        cv2.imshow('Frame', frame) 
        # Press Q on keyboard to exit. 
        # waitkey is how long to show in ms frame
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break
        #elif cv2.waitKey(10) == ord(' '):
        #    y_centers = []
        #    count = '0'
        #    print('reset')

    # Break the loop 
    else:  
        break
        
# When everything done, release  
# the video capture object
cap.release()
# Closes all the frames 
cv2.destroyAllWindows()

Exception in thread Thread-113:
Traceback (most recent call last):
  File "C:\Users\manny\anaconda3\envs\tensorflow\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-3-66455fb45092>", line 13, in run
    tts_engine.runAndWait()
  File "C:\Users\manny\anaconda3\envs\tensorflow\lib\site-packages\pyttsx3\engine.py", line 177, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started



### Post Review

In [ ]:
# apply a savgol filter to smooth out shakiness
df = pd.DataFrame({"Y Filt":savgol_filter(y_centers,15,2), 
                   "Y": y_centers})

peaks, _ = find_peaks(df['Y Filt'])

fig = go.Figure()


fig.add_trace(go.Scatter(x=df.index, y=df['Y Filt'],
                    mode='lines',
                    name='Y-Data'))


fig.add_trace(go.Scatter(x=df.index, y=df['Y'],
                    mode='lines',
                    name='Y-Raw'))

fig.add_trace(go.Scatter(x=df['Y Filt'].iloc[peaks].index, 
                         y=df['Y Filt'].iloc[peaks],
                    mode='markers',
                    name='Peaks'))

fig.update_layout(
    width=800,
    height=600,)

fig.show()
#plt.plot(x,y_centers_filt)
#plt.plot(peaks, y_centers_filt[peaks], "x")
#plt.show()

### To determine HSV Values of the Ball

In [15]:
#Lets view a particular frame
frame_number = 2
backSub = cv2.createBackgroundSubtractorKNN()

#define the hsv range
colorLower = np.array([24,80,20])
colorUpper = np.array([30,255,255])

In [19]:
cap = cv2.VideoCapture(video_path)
cap.set(1, frame_number)
ret, frame = cap.read()

#background subtract
fgMask = backSub.apply(frame)
#bitwise multiplication to grab moving part
new_frame = cv2.bitwise_and(frame,frame, mask = fgMask)
        
        
# identify the ball
mask = ball_finder(new_frame, colorLower, colorUpper)

#canny edge detection
edges = cv2.Canny(mask,100,200)
# find contours of the ball
image, contours, hierarchy = cv2.findContours(edges, 
                               cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


c = max(contours, key=cv2.contourArea)
M = cv2.moments(c)
center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
cv2.circle(frame, center, 10, (0,255,0), -1)
frame = cv2.drawContours(frame, contours, -1, (0,255,0), 3)
    
#write frame number in top left
cv2.rectangle(frame, (10, 2), (100,20), (255,255,255), -1)
cv2.putText(frame, str(cap.get(cv2.CAP_PROP_POS_FRAMES)), (15, 15),
       cv2.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))

cv2.imshow("Frame" + str(frame_number), frame)
cv2.waitKey(0)
cv2.destroyAllWindows()